# 案例要求


把 “人员名片信息.xlsx”中所有的人员，按照“名片模板.docx”文件格式生成的名片

<table width="100%" border="0">
<tr>
  <td valign="top" style="vertical-align:top; text-align:left;background-color:#ddffdd;">
 
 **<font color="blue" size="3px"> 名片模板.docx（Word）</font>**
 
<img src="images/名片模板.png" align="center" style="width:40%"/>
  </td>
  </tr>
    <tr>
  <td style="vertical-align:top; text-align:left;background-color:#ddffdd">

**<font color="blue" size="3px"> 人员名片信息.xlsx（Excel）</font>**
 
<img src="images/人员名片信息.png" align="center"/>

  </td>
 </tr>
</table>

In [ ]:
# 导入工具包
import docx
from docx import Document
import copy # 复制模板用
import xlrd

In [ ]:
# 为了防止综合案例的课程产生的文件过多，我们要对文件进行分类管理
# 本课程相关的文件都放到 files_dir 指定的目录下
files_dir = "02_files"

# 建立几个函数

本次课程的代码逻辑比前期课程的要多，我们建立几个函数的目的就是为了方便代码的阅读。

## 新增函数(1) get_namecard_data：从Excel中读取人员名片信息

In [ ]:
def get_namecard_data(xls):
    """
    定义函数：从Excel文件中读取名片数据
    :param xls: Excel 文件
    :return: 返回了 2 组数据
             1）名片字段列表：['姓名', '职位', '部门', '电话', '手机', '邮箱']
             2）名片信息列表（列表中的列表）
             [['张三', '开发工程师', '技术部'..],['李四', '业务经理', ...]]
    """
    # 打开文件
    workbook = xlrd.open_workbook(xls)
    # 获取第一个工作表，从0开始
    sheet = workbook.sheet_by_index(0)
    # 名片信息列表（列表中的列表）
    namecard_infos_list = []
    # 从 0 开始
    field_list = sheet.row_values(0)
    for row_index in range(1, sheet.nrows):
        namecard_infos_list.append(sheet.row_values(row_index))
    # 返回了 2 组数据
    #   1）名片字段列表：['姓名', '职位', '部门', '电话', '手机', '邮箱']
    #   2）名片信息列表（列表中的列表）
    #      [['张三', '开发工程师', '技术部'..],['李四', '业务经理', ...]]
    return field_list, namecard_infos_list

## 测试.1 get_namecard_data

用一个变量 namecards 来接收 get_namecard_data 返回的2组数据，看看结果

In [ ]:
staff_namecard_info_file = f"./{files_dir}/人员名片信息.xls"

# 获取的人员名片信息虽然有2组返回值，我们还是可以只赋值给一个变量
namecards = get_namecard_data(staff_namecard_info_file)
# 打印的格式为：( [ ....],[....] ) 最外面是 ** 圆括号 **
print(namecards) 

### 方括号是列表，圆括号是元组

In [ ]:
# 列表 list , 元组 tuple
my_list = list()   # 简写语法：my_list = []
my_tuple = tuple() # 简写语法：my_tuple = ()
print("列表:",my_list)  # 前期课程已经讲过
print("元组:",my_tuple) # 本次课程不做要求，知道元组就可以了，平常编程用 list 基本上可以满足 

## 测试.2 get_namecard_data

用两个变量 namecard_field_list, namecard_infos_list 来接收 get_namecard_data 返回的2组数据，看看结果

In [ ]:
# 获取的人员名片信息有2组返回值，我们**建议赋值给2个变量**
namecard_field_list, namecard_infos_list = get_namecard_data(staff_namecard_info_file)
print("名片字段：",namecard_field_list)
print("人员名片信息：",namecard_infos_list)

## 查看名片模版文件结构

In [ ]:
from docx_ext import read_docx

docx_template_file = f"./{files_dir}/Word_名片模版.docx"
# 查看一下“Word_名片模版.docx”模版中的文字块数据
read_docx(docx_template_file)
# 可以看到文字块包含 [姓名] [职位] [部门] [电话] [手机] [邮箱]

<font color=red>本次课程的目的就是将这些文字块中的 [姓名] [职位] [部门] [电话] [手机] [邮箱] 替换为下面 Excel 的数据</font>

<img src="images/人员名片信息.png" align="left"/>

## 新增函数(2) gen_replace_dict：将替换列表改为替换字典

In [ ]:
def gen_replace_dict(namecard_infos):
    """
    把列表替换为字典：
    ['刘德泽', '开发工程师'....] => {'[姓名]':'刘德泽', '[职位]':'开发工程师'....}
    :param namecard_infos: 名片信息列表，格式为：['刘德泽', '开发工程师'....] 
    :return:  字典格式为：{'[姓名]':'刘德泽', '[职位]':'开发工程师'....}
    """
    replace_dict = {} # 创建一个空字典
    for idx, namecard_info in enumerate(namecard_infos):
        # namecard_field_list = ['姓名', '职位', .....]
        field = namecard_field_list[idx]    # field = 姓名
        param = "[%s]" % field              # param = [姓名]
        replace_dict[param] = namecard_info # [姓名] = 刘德泽
    return replace_dict

## 测试 gen_replace_dict

In [ ]:
namecards = ['刘德泽', '开发工程师', '技术部', '87646000', '13399999999', 'ldz@abcde.com.cn']
# 测试 gen_replace_dict
replace_dict = gen_replace_dict(namecards)
print("字段信息：",namecard_field_list)
print("名片信息：",namecards)
print("字典信息：",replace_dict)

## 增加两个知识点

## replace 函数 

In [ ]:
# 定义一个模板
message_tpl = "[姓名]您好！欢迎学习[课程]"

# 进行第一次替换
message = message_tpl.replace("[姓名]", "刘德泽")
print("1)",message)

# 进行第二次替换
message = message.replace("[课程]", "Python")
print("2)",message)

## dict 字典

In [ ]:
# 定义一个空字典的语法
trans = dict() #  简写语法：trans = {}
print(trans)

# 给字典加入元素，比如：中英文翻译
trans["hello"] = "你好"
trans["bye"] = "再见"

print("字典数据：",trans)

In [ ]:
# 循环遍历 trans.keys()
for en in trans.keys():
    cn = trans[en] # 根据 en ,获取中文
    print("英文: %s => 中文: %s" % (en, cn))

## 新增函数(3) replace_run：替换文字块内容

In [ ]:
def replace_run(run, replace_dict):
    """
    用于替换 run 对象中的值，比如：[姓名] => 刘德泽
    :param run: 需要进行替换的文字块
    :param replace_dict: 替换字典，格式为：{'[姓名]':'刘德泽', '[职位]':'开发工程师'....}
    :return:
    """
    for key in replace_dict.keys():
        # print(key, replace_dict[key])
        # 比如：key = [姓名] 
        # 如果 文字块 run 中包含有 [姓名]，则进行替换 
        if key in run.text:  
            run.text = run.text.replace(key, replace_dict[key])

## 测试 replace_run

In [ ]:
doc = Document()
print("替换字典 =>",replace_dict)
p1 = doc.add_paragraph('姓名：[姓名]')
p1.add_run("  职位：[职位]")

print("段落替换前 =>",p1.text)
# 测试 replace_run
replace_run(p1.runs[0],replace_dict)
print("段落替换后 =>",p1.text)

## 新增函数(4) replace_doc：遍历替换文档的段落

In [ ]:
def replace_doc(document, replace_dict):
    """
    替换文档中的 [姓名] [职位] 为 指定的名片信息
    :param document: 包含 [姓名] [职位]...的文档模板对象
    :param replace_dict: 
    :return:
    """
    # 循环遍历段落
    for paragraph in document.paragraphs:
        for run in paragraph.runs:
            replace_run(run, replace_dict)

    return document

## 新增函数(5) gen_namecard_main：名片批量生成主函数

In [ ]:
def gen_namecard_main(docx_tpl):
    """
    名片批量生成的主函数
    docx_tpl：名片模版文件路径
    """
    # 获取文档对象（作为名片模版）
    name_card_tpl = docx.Document(docx_tpl)

    # 循环遍历名片信息列表（多个人员的名片数据）
    for namecard_info in namecard_infos_list:
        # 每次都要复制一份模板，防止原始的模板被替换了
        name_card_doc = copy.deepcopy(name_card_tpl)
        # 开始替换文档内容（参数分别为：docx模板副本 和 当前员工的名片数据）
        # namecard_info=['刘德泽', '开发工程师', '技术部', '87646000', '13399999999', 'ldz@abcde.com.cn']
    #     replace_doc(name_card_doc, namecard_info)
        replace_dict = gen_replace_dict(namecard_info)
        replace_doc(name_card_doc,replace_dict )
        # 获取人员姓名
        staff_name = namecard_info[0]
        name_card_doc.save(f'./{files_dir}/{staff_name}的名片.docx')

# 直接调用主函数，参数为：名片模版文件 
gen_namecard_main(docx_template_file)

# Python总结

1、元组的了解  
```
my_tuple = tuple()

my_tuple = ()
```
2、字典的用法
```
my_dict = dict()

my_dict = {}
```

<font color=red>大家注意括号不要记混了： 列表 [ ]、元组 ( )、字典 { } </font>

3、创建函数可以提高代码的阅读性

* 可以看到每个函数实现的功能比较单一，代码量都比较少，这样代码易读性就好的多；

* 另外，创建的函数还可以重复利用多次；
```
比如：下面思考题中会重复再用一次 gen_namecard_main() 生成名片的主函数，只是传入的参数（模板文件）不同
```
* 如果把所有的逻辑写在一起，代码肯定很长了，大家可以尝试不用函数，把所有的逻辑写在一起；


# 思考题

运行如下代码，“Word_名片模版_思考题.doc”模板中有一个段落中的内容 [职位] 有3个文字块，刚才的代码就无法正确生成名片了，怎么办？

这个思考题有点难度，能解决的朋友，就非常厉害了，不能解决也没有关系，只要思考了，思维能力也一定能得到锻炼的，加油！

In [ ]:
# 运行查看“Word_名片模版_思考题.docx”，可以看到 “[职位]” 属于3个文字块，无法成功替换
thinking_tpl_file = f"./{files_dir}/Word_名片模版_思考题.docx"
# 查看一下“Word_名片模版_思考题.docx”模版中的文字块数据
read_docx(thinking_tpl_file)

In [ ]:
# 用“Word_名片模版_思考题.docx” 生成名片看看，会出现什么问题？
gen_namecard_main(thinking_tpl_file)